In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import re
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

import helpers as hp

%load_ext autoreload
%autoreload 2


# Compute Auckland and Wellington fares through their web APIs


In [11]:
def get_journey_auckland(orig, dest, departure_time=None):
    """
    INPUT
    ------
    orig : list
        WGS84 longitude-latitude pair
    dest : list
        WGS84 longitude-latitude pair
    departure_time : string
        ISO 8601 datetime; e.g. '2017-06-01T07:30:00'
        
    OUTPUT
    ------
    dictionary
        Decoded JSON response of journey
    """
    url = 'https://api.at.govt.nz/v2/public-restricted/journeyplanner/silverRailIVU/plan'
    fromLoc ='{!s},{!s}'.format(orig[1], orig[0])
    toLoc ='{!s},{!s}'.format(dest[1], dest[0])
    if departure_time is None:
        departure_time = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    date = departure_time + '+12:00'  # Add UTC offset
    params = {
        'from': 'from',
        'to': 'to',
        'fromLoc': fromLoc,
        'toLoc': toLoc,
        'timeMode': 'A',
        'date': date, 
        'modes': 'BUS,TRAIN,FERRY',
        'operators': '',
        'optimize': 'QUICK',
        'maxWalk': 2000,
        'maxChanges': '-1',
        'routes': '',
        'subscription-key': '323741614c1c4b9083299adefe100aa6',
    }
    r = requests.get(url, params=params)
    
    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def get_fare_auckland(journey):
    """
    Given a journey of the form output by :func:`get_journey_auckland`, 
    return the journey's adult fare (float)'
    """
    try:
        fare = journey['response']['itineraries'][0]['fareAdult']/100
    except:
        fare = None
    return fare

def get_journey_wellington(orig, dest, departure_time=None):
    """
    INPUT
    ------
    orig : list
        WGS84 longitude-latitude pair
    dest : list
        WGS84 longitude-latitude pair
    departure_time : string
        ISO 8601 datetime; e.g. '2017-06-01T07:30:00'
        
    OUTPUT
    ------
    text
        HTML response of journey query
    """
    url = 'https://www.metlink.org.nz/journeyplanner/JourneyPlannerForm'
    from_coords ='{!s},{!s}'.format(orig[1], orig[0])
    to_coords ='{!s},{!s}'.format(dest[1], dest[0])
    
    if departure_time is None:
        departure_time = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    date, time = departure_time.split('T')
        
    params = {
        'From': 'from',
        'To': 'to', 
        'Via': '',
        'When': 'LeaveAfter',
        'Date': date,
        'Time': time,
        'MaxChanges': 5,
        'WalkingSpeed': 4,
        'MaxWalking': 2000,
        'Modes[Train]': 'Train',
        'Modes[Bus]': 'Bus',
        'Modes[Ferry]': 'Ferry',
        'Modes[Cable+Car]': 'Cable Car',
        'ShowAdvanced': '',
        'FromCoords': from_coords,
        'ToCoords': to_coords,
        'ViaCoords': '',
        'action_doForm': 'Go',
    }
    r = requests.get(url, params=params)
    # Raise an error if bad request
    r.raise_for_status()
    return r.text         

# Estimate Wellington card fare discount
path = hp.DATA_DIR/'wellington'/'transit_fares.csv'
f = pd.read_csv(path)
f['card/cash'] = f['card_fare']/f['cash_fare']
r = f['card/cash'].mean()
print('estimated Wellington card discount rate=', r)

def get_fare_wellington(journey, card_discount=r):
    """
    Given a journey of the form output by :func:`get_journey_wellington`, 
    extract the journey's adult cash fare (float), multiply it by the given
    discount rate to estimate the adult card fare, and return the result.
    """
    pattern = 'Total adult fare </span><strong>&#36;(\d+\.\d\d)</strong>'
    m = re.search(pattern, journey)
    if m:
        fare = float(m.group(1))
    else:
        fare = None
    return round(r*fare, 2)

def collect_fares(rental_points, departure_time, region):
    """
    """
    # Get all pairs of points excluding equal points
    f = rental_points[['rental_area', 'geometry']].copy()
    rows = [[o[0], o[1].coords[0], d[0], d[1].coords[0]] for o, d in product(f.values, f.values) if o[0] != d[0]]
    f = pd.DataFrame(rows, columns=['orig_name', 'orig', 'dest_name', 'dest'])

    if region == 'auckland':
        get_journey = get_journey_auckland
        get_fare = get_fare_auckland
        time_per_call = 3.6
    elif region == 'wellington':
        get_journey = get_journey_wellington
        get_fare = get_fare_wellington
        time_per_call = 2.4
    
    print('This will take about {:02f} minutes'.format(f.shape[0]*time_per_call/60))

    # Get journeys for each pair
    rows = []
    for __, row in f.iterrows():
        try:
            j = get_journey(row['orig'], row['dest'], departure_time=departure_time)
            fare = get_fare(j)
        except:
            fare = None
        rows.append([row['orig_name'], row['dest_name'], fare])

    g = pd.DataFrame(rows, columns=['orig_name', 'dest_name', 'card_fare'])
    return g


estimated Wellington card discount rate= 0.782401654073


In [ ]:
# Test some
orig = [174.7433853149414, -36.85517522550505]
dest = [174.79625701904297, -36.82550066651677]
%time j = get_journey_auckland(orig, dest)
j
get_fare_auckland(j)

orig = (174.7708511352539,-41.28394744513899)
dest = (174.78861808776855,-41.297458248607995)
%time r = get_journey_wellington(orig, dest, departure_time='2017-06-01T07:30:00')
r
get_fare_wellington(r)

In [4]:
regions = ['auckland', 'wellington']
for region in regions:
    rp = get_data(region, 'rental_points')
    departure_time = '2017-06-01T07:30:00'
    g = collect_fares(rp, departure_time, region)

    path = get_path(region, 'transit_costs')
    g.to_csv(str(path), index=False)
    print('* ', region)
    print(g.head())


This will take about 72.240000 minutes
*  wellington
  orig_name                  dest_name  fare
0  Brooklyn  Carterton/South Wairarapa   NaN
1  Brooklyn               Eastern Bays  14.5
2  Brooklyn               Epuni/Avalon  12.5
3  Brooklyn                   Hataitai  10.5
4  Brooklyn    Heretaunga/Silverstream  16.5


# Canterbury has no fare calculator API. So estimate Canterbury fares from fare zones and fare table.

In [2]:
# Fares
fares = pd.DataFrame([[1, 2.55], [2, 3.75]], columns=['#zones_traveled', 'card_fare'])
fares

# Zones
path = DATA_DIR/'canterbury'/'fare_zones.geojson'
zones = gpd.read_file(str(path))
zones

# Attach zones to rental points
rp = hp.get_data('canterbury', 'rental_points')
g = gpd.sjoin(rp, zones, op='within')
g = g[['rental_area', 'zone']].copy()
g.head()

# Compute origin and destination zones
f = hp.get_data('canterbury', 'commutes_transit')
f = f.merge(g.rename(columns={'rental_area': 'orig_name', 'zone': 'orig_zone'}))
f = f.merge(g.rename(columns={'rental_area': 'dest_name', 'zone': 'dest_zone'}))

# Compute #zones traveled, then card fare
f['#zones_traveled'] = abs(f['orig_zone'] - f['dest_zone']) + 1
f = f.merge(fares)

# Cut down and save
f = f[['orig_name', 'dest_name', 'card_fare']].copy()
path = hp.get_path('canterbury', 'transit_costs')
f.to_csv(str(path), index=False)
f

,orig_name,dest_name,card_fare
0,Rangiora/Kaiapoi,Richmond/Avonside,3.75
1,Banks Peninsula/Selwyn,Richmond/Avonside,3.75
2,Styx/Parklands,Banks Peninsula/Selwyn,3.75
3,Sawyers Arms/Northcote/Belfast,Banks Peninsula/Selwyn,3.75
4,Marshland/Redwood,Banks Peninsula/Selwyn,3.75
5,Burnside/Harewood,Banks Peninsula/Selwyn,3.75
6,Bishopdale/Papanui,Banks Peninsula/Selwyn,3.75
7,St Albans North/ Mairehau,Banks Peninsula/Selwyn,3.75
8,Richmond/Shirley,Banks Peninsula/Selwyn,3.75
9,Burwood/Dallington/Avondale,Banks Peninsula/Selwyn,3.75
